# Create Heatmaps for Weather Parameters

We'll use Pandas to read our CSV file and create the locations and measurements from the DataFrame. We'll use gmaps and the API key to create heatmaps and the locations map, and we'll use the requests dependency to make a request to the Google Places JSON file. This will allow us to get hotel locations from the latitude and longitude of the city.

In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [2]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Marsh Harbour,BS,2020-11-29 03:16:26,26.54,-77.06,74.70,66,79,5.01
1,1,Pittsburg,US,2020-11-29 03:15:21,38.03,-121.88,52.00,49,0,1.45
2,2,Ushtobe,KZ,2020-11-29 03:16:26,45.25,77.98,25.48,95,48,11.34
3,3,Jamestown,US,2020-11-29 03:16:26,42.10,-79.24,35.01,59,1,9.17
4,4,Bitung,ID,2020-11-29 03:16:27,1.45,125.18,78.80,94,40,6.93


Note:
- using gmaps: The data we use for any mapping must be either an integer or a floating-point decimal number. Let's check the data types for the columns of our DataFrame

In [3]:
# Confirm data types for the columns we are going to use in the heatmaps we are going to create will work with gmaps
# make sure they're 'int' or 'float'
city_data_df.dtypes

City_ID         int64
City           object
Country        object
Date           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

## Create a Maximum Temperature Heatmap

In [4]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

### The general syntax for creating a heatmap:

In [5]:
# # 1. Assign the locations to an array of latitude and longitude pairs.
# locations = [latitude, longitude]
# # 2. Assign the weights variable to some values.
# temperatures = # an array of length equal to the locations array length
# # 3. Assign the figure variable to the gmaps.figure() attribute.
# fig = gmaps.figure()
# # 4. Assign the heatmap_layer variable to the heatmap_layer attribute and add in the locations.
# heatmap_layer = gmaps.heatmap_layer(locations, weights=temperatures)

# # 5. Add the heatmap layer.
# fig.add_layer(heatmap_layer)
# # 6. Call the figure to plot the data.
# fig

We will get an `InvalidWeightException` error if we have any negative numbers. Therefore we need to remove any temperatures less than 0.

To remove these values in our DataFrame, we can use a `for` loop to iterate through `max_temp` and add temperatures that are greater than 0 toa new list

In [6]:
# # Get the maximum temperature.
# max_temp = city_data_df["Max Temp"]
# temps = []
# for temp in max_temp:
#     temps.append(max(temp,0))
# # we're using the max() function to get the largest value between the temp and 0. If the temp is less than 0, then 0 will be added to the list in its place. Otherwise, the temp is added to the list

Or use list comprehension instead of the for loop above in the heat_layer line:

In [7]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
# Assign the figure variable.
fig = gmaps.figure()
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in max_temp])
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

## Adjust Heatmap Zoom, Intensity, and Point Radius

In [8]:
# Heatmap of temperature - adjusted zoom_level and centering
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
# Assign the figure variable.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in max_temp])
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

### Adjust the dissipation option for heatmaps

The options for this are:

1. The default option for the dissipation is "True," so we need to set our "dissipation" to "False."
2. We can add max_intensity to make each measurement have a better gradient variance.
3. We can add point_radius to make each measurement radius larger.

In [9]:
# Heatmap of temperature - adjust for intensity, dissipation, and point radius in the gmaps.heal_layer() attribute
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
# Assign the figure variable.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in max_temp], dissipating=False, max_intensity=300, point_radius=4)
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

Doing this increases the size of the circles and the gradients show differences

## Create a Percent Humidity Heatmap

In [10]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

## Create a Percent Cloudiness Heatmap

In [11]:
# Heatmap of percent cloudiness
locations = city_data_df[["Lat", "Lng"]]
clouds = city_data_df["Cloudiness"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=clouds, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

## Create a Percent Wind Speed Heatmap

In [12]:
# Heatmap of percent Wind Speed
locations = city_data_df[["Lat", "Lng"]]
wind = city_data_df["Wind Speed"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=wind, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

# Get Vacation Criteria
We need to prompt the user to enter the minimum and maximum temperature ranges as floating-point decimal numbers to filter the city_data_df DataFrame.

In [13]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [14]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
4,4,Bitung,ID,2020-11-29 03:16:27,1.45,125.18,78.80,94,40,6.93
7,7,Iracoubo,GF,2020-11-29 03:16:27,5.48,-53.20,78.31,90,51,6.29
9,9,Micheweni,TZ,2020-11-29 03:16:27,-4.97,39.83,79.48,78,83,3.18
10,10,Fare,PF,2020-11-29 03:16:27,-16.70,-151.02,82.76,71,13,5.73
15,15,Daru,PG,2020-11-29 03:16:28,-9.08,143.21,86.11,63,54,7.85
17,17,Hilo,US,2020-11-29 03:14:53,19.73,-155.09,80.60,57,1,9.17
18,18,Namatanai,PG,2020-11-29 03:16:28,-3.67,152.43,85.15,71,61,6.04
21,21,Butaritari,KI,2020-11-29 03:16:28,3.07,172.79,81.63,82,0,14.72
23,23,Vaini,TO,2020-11-29 03:16:28,-21.20,-175.20,82.40,74,75,21.92
27,27,Rikitea,PF,2020-11-29 03:16:29,-23.12,-134.97,76.75,81,100,11.41


In [15]:
# Check for null values in any of the rows
preferred_cities_df.count()

City_ID       120
City          120
Country       120
Date          120
Lat           120
Lng           120
Max Temp      120
Humidity      120
Cloudiness    120
Wind Speed    120
dtype: int64

Note:
1. Depending on the time of year and the seasons, you might have to adjust the minimum and maximum temperature to get enough cities.
2. It is a good idea to keep the number of cities to fewer than 200 to make it easier to plot the markers on the heatmap.
3. If you have some rows with null values, you'll need to drop them using the dropna() method at the end of your filtering statement when you are creating the new DataFrame.

# Map Vacation Criteria
Using the coordinates from the `preferred_cities_df` DataFrame, find a hotel using our Google Places API and then retrieve that hotel information. Once we retrieve the hotel information, we'll need to store it so we can reference it and add the information to the pop-up marker.

## Get Travel Destinations
We don't want to add the hotel info to the filtered `preferred_cities_df` DataFrame, as the criteria for this df will always change. So we will want to make a copy of the DataFrame using the .copy() method and create a heatmap and pop-up markers

In [16]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
4,Bitung,ID,78.80,1.45,125.18,
7,Iracoubo,GF,78.31,5.48,-53.20,
9,Micheweni,TZ,79.48,-4.97,39.83,
10,Fare,PF,82.76,-16.70,-151.02,
15,Daru,PG,86.11,-9.08,143.21,
17,Hilo,US,80.60,19.73,-155.09,
18,Namatanai,PG,85.15,-3.67,152.43,
21,Butaritari,KI,81.63,3.07,172.79,
23,Vaini,TO,82.40,-21.20,-175.20,
27,Rikitea,PF,76.75,-23.12,-134.97,


Using the latitude and longitude and specific parameters, use the Google Places Nearby Search request to retrieve a hotel and add it to the Hotel Name column.

## Retrieve Hotels from a Nearby Search
### Set the Parameters for a Nearby Search

In [17]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}


### Iterate Through hotel_df DataFrame

In [31]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [32]:
hotel_df

,City,Country,Max Temp,Lat,Lng,Hotel Name
4,Bitung,ID,78.80,1.45,125.18,Peoplesight Learning Center ( PLC )
7,Iracoubo,GF,78.31,5.48,-53.20,Le Gite D Iracoubo
9,Micheweni,TZ,79.48,-4.97,39.83,
10,Fare,PF,82.76,-16.70,-151.02,Maitai Lapita Village Huahine
15,Daru,PG,86.11,-9.08,143.21,New century hotel
...,...,...,...,...,...,...
571,Aden,YE,78.73,12.78,45.04,Sama Emirate Hotel
573,Mumford,GH,77.52,5.26,-0.76,Spanish Villa
574,Pantai Remis,MY,80.60,4.46,100.63,Leisure Homestay
578,Puerto Escondido,MX,79.54,15.85,-97.07,Aldea del Bazar Hotel and Spa


# Create a Maximum Temperature Heatmap from a Hotel DataFrame

In [29]:
# Add a heatmap of temperature for the vacation spots.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

Add markers
the syntax for markers is
`markers = gmaps.marker_layer(marker_locations)`, where the `marker_locations` are latitudes and longitudes.

In [30]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

### Add an info box template for the pop-up marker for each city that displays the hotel name/city name/country/max temp

In [33]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

This is in HTML
First, the info_box_template variable is assigned to a multiline string using three quotes. The text inside the multiline string is HTML code. HTML code is defined by the opening and closing the angular brackets (e.g., <tag> and <tag/>). Angular brackets always come in pairs. The opening angular bracket is followed by some text inside, such as dl, dt, and dd. The closing angular bracket is preceded by a forward-slash ("/"). The text inside with the angular brackets is called a tag.

    Here's what these tags mean:

- The `<dl>` tag is a description list (dl).
- The `<dt>` tag is a term or name in a description list that is nested under the `<dl>` tag.
- The `<dd>` tag is used to define the term or name or `<dt>` tag.
- If we were to write out these tags on paper, it would look like this.

- Description List: `<dl>`
    - Description Term: `<dt>`
        - Description Definition: `<dd>`
    - Description Term: `<dt>`
        - Description Definition: `<dd>`

Next, add the data to the code by iterating through the hotel_df DataFrame using the `iterrows()` function. Then add the information to the `gmaps.marker_layer()` attribute with the locations.

In [34]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [35]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))